## Background

In the previous notebook, we experienced that the data we wanna access are loaded in a form called **`xarray.dataset`**. This is the form in which earth observation data are usually stored in a datacube. Understanding the structure of a **`xarray.dataset`** is the key to enable us work with these data. Thus, in this notebook, we are mainly dedicated to helping users of our datacube understand its data structure.

Firstly let's come to the end stage of the previous notebook, where we have loaded a data product. The data product "s2_l2a_bavaria" is used as example in this notebook.

In [4]:
import datacube
# To access and work with available data

import pandas as pd
# To format tables

from odc.ui import DcViewer 
# Provides an interface for interactively exploring the products available in the datacube

from odc.ui import with_ui_cbk
# Enables a progress bar when loading large amounts of data.

import xarray as xr

import matplotlib.pyplot as plt

# Set config for displaying tables nicely
# !! USEFUL !! otherwise parts of longer infos won't be displayed in tables
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", None)

# Connect to DataCube
# argument "app" --- user defined name for a session (e.g. choose one matching the purpose of this notebook)
dc = datacube.Datacube(app = "nb_understand_ndArrays", config = '/home/datacube/.datacube.conf')

# Load Data Product
ds = dc.load(product = "s2_l2a_bavaria",
             measurements = ["blue", "green", "red"],
             longitude = [12.493, 12.509],
             latitude = [47.861, 47.868],
             time = ("2018-04-01", "2019-03-31"))

print(ds)

<xarray.Dataset>
Dimensions:      (time: 269, x: 124, y: 84)
Coordinates:
  * time         (time) datetime64[ns] 2018-04-01T10:00:22 ... 2019-03-30T10:...
  * y            (y) float64 5.308e+06 5.308e+06 ... 5.307e+06 5.307e+06
  * x            (x) float64 7.612e+05 7.612e+05 ... 7.624e+05 7.624e+05
    spatial_ref  int32 25832
Data variables:
    blue         (time, y, x) int16 0 0 0 0 0 0 0 ... 518 515 519 525 499 473
    green        (time, y, x) int16 0 0 0 0 0 0 0 ... 756 752 736 760 738 726
    red          (time, y, x) int16 0 0 0 0 0 0 0 ... 428 433 450 469 448 433
Attributes:
    crs:           EPSG:25832
    grid_mapping:  spatial_ref


![xarray data structure](https://live.staticflickr.com/65535/51083605166_70dd29baa8_k.jpg)

The figure above is a diagramm of the structure of the **`xarray.dataset`** we've just loaded. Combined with the diagramm, we hope you may better interpret the explanation below of the data strucutre.

As read from the output block, this dataset has three ***Data Variables*** , "blue", "green" and "red" (shown with colors in the diagramm). It refers to the spectral bands and can be defined with `measurements` argument in the `load` function.

Ultimately, each data variable is a **multi-dimensional *Data Array*** of same structure; in this case, it is a **three-dimensional array** (shown as 3D Cube in the diagramm). `time`, `x` and `y` are the ***Dimensions*** (shown as axis along each cube in the diagramm), which are defined with argument `time`, `longitude` and `latitude` respectively in the `load` function.

In this dataset, there are 269 ***Coordinates*** under `time` dimension, which means there are 269 steps along the `time` axis (as shown in the diagramm). Same rule again, there are 124 coordinates under `x` dimension and 84 coordinates under `y` dimension, indicating that there are 124 pixels along `x` axis and 84 pixels along `y` axis.

As for the term ***Dataset***, it's just a *Container* holding all the multi-dimensional arrays of same structure (shown as the red-lined box holding all 3D Cubes in the diagramm).

Thus we know our dataset has a spatial extent of 124 by 84 pixels within given lon/lat range, spans over 269 time stamps at each spectral band.

Now let's deconstruct the dataset we've just loaded to have things (hopefully) more cleared!:D

## 2 Core Data Structures of "xarray"

### I. `DataArray`
* **Labeled, N-dimensional array**
* based on **`pandas.Series`**

Its Key Properties:
* **Data Variables (`values`)**: A `numpy.ndarray` holding values *(e.g. reflectance values of spectral bands)*.
* **Dimensions (`dims`)**: Dimension names for each axis *(e.g. 'x', 'y', 'time')*.
* **Coordinates (`coords`)**: Coordinates of each value along each axis *(e.g. longitudes along 'x'-axis, latitudes along 'y'-axis, datetime objects along 'time'-axis)*
* **Attributes (`attrs`)**: A dictionary(`dict`) containing Metadata.


### II. `Dataset`
* N-dimensional array database
* **Container of `DataArray`**
* **Dictionary-like**
* Analog to **`pandas.DataFrame`**

Having these two data structures, "xarray" is able to
* **Pull out arrays by name**
* **Select or Combine data along a dimension across all array simultaneously** *(e.g. Select NIR-band images from time stamps 1-12, assuming there're 12 time stamps in total, for an area X; Then calculate the median of 12 time stamps for each pixel?)*

![RGB](https://live.staticflickr.com/65535/51014414875_615bb016ea_k.jpg)

Let's look at the structure of he graphic above depicts, 

In [51]:
blue = ds["blue"][1][979]#[500:501]

print(len(blue))
blue

501


<xarray.DataArray 'blue' (x: 501)>
array([ 7296,  7100,  7760,  7560,  6196,  5252,  6988,  6000,  4120,
        5160,  5936,  4452,  6144,  6756,  5936,  4940,  5208,  4684,
        4740,  6308,  7460,  5660,  5040,  8840, 10808,  9728,  8728,
        7872,  7116,  7204,  6616,  6392,  6664,  6564,  6504,  6344,
        6344,  6784,  6816,  7040,  7140,  7260,  7360,  7368,  7296,
        7164,  7144,  7020,  7052,  7272,  7372,  7060,  6624,  7344,
        5612,  6020,  7992,  7376,  7464,  8248,  8104,  7988,  6684,
        5228,  6304,  7880,  7332,  6644,  6276,  6484,  6508,  8188,
        7696,  7060,  7232,  7772,  7772,  6600,  6860,  7504,  7784,
        7896,  6756,  7016,  7948,  6984,  6992,  7432,  7364,  7436,
        7420,  7656,  7616,  7668,  7956,  8384,  8472,  8140,  7896,
        7588,  7760,  7564,  7676,  7536,  7272,  7272,  7000,  6744,
        6376,  6100,  6104,  6220,  6200,  4964,  2126,  1854,  1980,
        2132,  2002,  2402,  2248,  2104,  2084,  2160,  2114,  2120,
        2156,  2762,  4028,  4652,  4928,  4716,  5400,  5348,  4584,
        6004,  2634,  1514,  2530,  2938,  4572,  6208,  4008,  4552,
        5304,  3082,  1948,  2036,  2710,  5124,  3938,  1848,  1908,
        2612,  3420,  3482,  6100,  4004,  1662,  1534,  1930,  1934,
        1708,  1338,  1188,  1128,  1680,  1416,  1396,  1652,  1550,
        2040,  1678,  1962,  2562,  3376,  4296,  4124,  4432,  5160,
...
        3332,  3732,  4672,  5756,  6276,  7236,  8168,  8224,  7552,
        7440,  6452,  6364,  7892,  7764,  8416,  7456,  6036,  6020,
        5744,  4264,  3444,  3428,  3392,  3274,  3320,  3786,  4488,
        4344,  4688,  4704,  3996,  3306,  3958,  4296,  5224,  5224,
        4840,  4920,  4064,  3906,  3852,  3958,  3908,  4528,  5264,
        5028,  5156,  6148,  4680,  2944,  2830,  2590,  2784,  2426,
        1556,  1456,  1408,  1332,  1318,  1578,  1322,  1072,  1080,
        1044,  1498,  1172,   849,   720,   694,   681,   735,   810,
         904,   959,  1058,  1338,  1626,  1752,  1946,  2238,  2498,
        2632,  2558,  2522,  2524,  2474,  2432,  2514,  2596,  2574,
        2588,  2520,  2348,  2216,  2078,  1950,  2098,  1864,  1936,
        1878,  1570,  1430,  1316,  1294,  1222,  1158,  1076,  1023,
        1034,  1114,  1356,  1406,  1562,  1420,  1294,  1194,  1001,
        1060,  1042,  1084,   952,   924,   929,   915,   813,   794,
         753,   909,   891,  1184,  1116,   946,   868,   773,   795,
         943,   849,   794,   813,   766,   761,   785,   925,   905,
         911,   919,  1018,  1118,   980,   858,   817,   993,   792,
         816,   745,   723,   695,   663,   687,   622,   587,   523,
         511,   458,   377,   286,   257,   241,   203,   206,   211,
         190,   168,   157,   143,   132,   128], dtype=int16)
Coordinates:
    time         datetime64[ns] 2020-04-03T10:17:35
    y            float64 5.269e+06
  * x            (x) float64 7.965e+05 7.965e+05 ... 8.015e+05 8.015e+05
    spatial_ref  int32 25832
Attributes:
    units:         reflectance
    nodata:        0
    crs:           EPSG:25832
    grid_mapping:  spatial_ref

Now if you feel you understand the structure of multi-dimensional data, we believe you are ready to learn how to grasp the subset of data you need for your analysis.